In [1]:
%matplotlib inline
import math
import matplotlib.pyplot as plt
import pandas as pd
import FukudaCsv


# アラームログが格納されているディレクトリを指定する. 空文字を指定した場合はサンプルデータを表示します．
dir_path_to_process = ''

# CSVのロード
df = FukudaCsv.load(dir_path_to_process)

記録時刻:2021-10-12 07:38:00 〜 2021-10-14 17:06:00,  2 days 09:28:00 (57時間)


In [2]:
df.head()

,Unnamed:0,No.,時刻,床番号,コード,発生要因,値,継続時間,レベル,継続時間int,hour,dayhour,dayhour10min,alarm_level,alarm_type,start_datetime,diff_datetime,diff_sec,diff_min
0,44933,1798,2021-10-12 07:38:00,CH1001,0x201F,心電図ノイズ混入,---,4,N(TECH_),4,07,10月12日07時,12日07時30分台,N,(TECH_),2021-10-12 07:38:00,0 days 00:00:00,0.0,0
1,44932,1799,2021-10-12 07:38:00,CH1001,0x201F,心電図ノイズ混入,---,5,N(TECH_),5,07,10月12日07時,12日07時30分台,N,(TECH_),2021-10-12 07:38:00,0 days 00:00:00,0.0,0
2,44931,1800,2021-10-12 07:49:00,CH1001,0x201F,心電図ノイズ混入,---,3,N(TECH_),3,07,10月12日07時,12日07時40分台,N,(TECH_),2021-10-12 07:38:00,0 days 00:11:00,660.0,11
3,4956,2983,2021-10-12 08:09:00,CH1002,0x1010,ＳｐＯ2下限,(計測値)89 < (下限閾値)90,25,H(MEAS_),25,08,10月12日08時,12日08時00分台,H,(MEAS_),2021-10-12 07:38:00,0 days 00:31:00,1860.0,31
4,44906,1825,2021-10-12 08:11:00,CH1001,0x201F,心電図ノイズ混入,---,8,N(TECH_),8,08,10月12日08時,12日08時10分台,N,(TECH_),2021-10-12 07:38:00,0 days 00:33:00,1980.0,33


In [3]:
# コードと発生要因の一覧を得る. 解析に不要な情報があれば削除する際に必要になる．
# df.groupby(['コード', '発生要因', 'alarm_level'])[['発生要因']].count()

In [4]:
# アラームレベルが H なデータのみを得る
df = df.query('alarm_level == "H"')

# df.head()

In [5]:
# アラーム記録開始時刻から終了時刻まで１秒一行としたDataFrameを作成する．
# isVitalAlarmRinging列に， アラームが鳴っていたら 1 を， 鳴っていなければ 0 を代入する. 0で初期化しておく．

start = df['start_datetime'].min()
end = df['時刻'].max()
timeseries = pd.date_range(start, end, freq='S')

df2 = pd.DataFrame(
    data = {
        '時刻' : pd.Series(timeseries),
        'isVitalAlarmRinging' : 0,
    }
)
df2.head()

,時刻,isVitalAlarmRinging
0,2021-10-12 07:38:00,0
1,2021-10-12 07:38:01,0
2,2021-10-12 07:38:02,0
3,2021-10-12 07:38:03,0
4,2021-10-12 07:38:04,0


In [6]:
# 上記で作成したDataFrameのisVitalAlarmRingingにアラームの鳴動状況を代入するため，
# アラーム履歴のDataFrameを一行ずつ取得し， 開始時刻から終了時刻（開始時刻 + 継続時間)分の行を1で埋める．

for index, row in df.iterrows():
    second_ring_from = int(row['diff_sec'])
    second_ring_to = second_ring_from + row['継続時間int']
    df2.loc[second_ring_from:second_ring_to, 'isVitalAlarmRinging'] = 1
    
df2.query("isVitalAlarmRinging == 1").head()

,時刻,isVitalAlarmRinging
1860,2021-10-12 08:09:00,1
1861,2021-10-12 08:09:01,1
1862,2021-10-12 08:09:02,1
1863,2021-10-12 08:09:03,1
1864,2021-10-12 08:09:04,1


In [7]:
# アラームが鳴っていた秒数となっていなかった秒数の合計をとり, 全体に対する割合を求める． 
df3 = df2.groupby(['isVitalAlarmRinging'])[['isVitalAlarmRinging']].count()
df3['totalMinutes'] = df3.apply(lambda x: math.floor(x['isVitalAlarmRinging'] / 60), axis=1)     #isVitalAlarmRingingの単位は秒なので 分 にした数値を格納
df3['totalHours'] = df3.apply(lambda x: math.floor(x['isVitalAlarmRinging'] / 3600), axis=1)     #isVitalAlarmRingingの単位は秒なので 時間 にした数値を格納
df3

,isVitalAlarmRinging,totalMinutes,totalHours
isVitalAlarmRinging,,,
0,121312,2021,33
1,85509,1425,23
